# Replication - Ecoregion 2.1

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [3]:
# Intersect the dissolved MPAs with the ecoregions

in_feature_1 = "WDPA_MPA_dis"
in_feature_2 = "MEco_mol_Carib"
out_feature = "\\MPA_dis_Eco_int_Replic2_1"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseIntersect(
    in_features=[in_feature_1,in_feature_2],
    out_feature_class=output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_dis_Eco_int_Replic2_1'>

In [18]:
# Repair geoemtry on intersect and calculate area of intersecting bits

in_feature = "MPA_dis_Eco_int_Replic2_1"
out_feature = "MPA_dis_Eco_int_singl_Replic2_1"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
    
    #make the intersected layer signlepart to ensure all polygons are individual
    arcpy.management.MultipartToSinglepart(
        in_features= in_feature,
        out_feature_class= out_feature
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features=out_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )

repair_geo_area()


In [19]:
# Only keep features with an area greater than .24 km2

# Select features with an area greater than .24 km2 and make a new layer from that and export it to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"

in_feature = "MPA_dis_Eco_int_singl_Replic2_1"
out_feature = "MPA_dis_Eco_int_singl_big_Replic2_1"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def select_big_excel():
    # make a layer of polygons with an area greater than .24 km2    
    arcpy.conversion.ExportFeatures(
        in_features= in_feature,
        out_features= out_feature,
        where_clause="area >= 0.24",
        use_field_alias_as_name="NOT_USE_ALIAS",
        field_mapping='FID_WDPA_MPA_dis "FID_WDPA_MPA_dis" true true false 4 Long 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,FID_WDPA_MPA_dis,-1,-1;FID_MEco_mol_Carib "FID_MEco_mol_Carib" true true false 4 Long 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,FID_MEco_mol_Carib,-1,-1;ECOREGION "ECOREGION" true true false 50 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,ECOREGION,0,50;REALM "REALM" true true false 40 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,REALM,0,40;PROVINC "PROVINC" true true false 254 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,PROVINC,0,254;TYPE "TYPE" true true false 50 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,TYPE,0,50;BIOME "BIOME" true true false 254 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,BIOME,0,254;MEOW_PPOW "MEOW_PPOW" true true false 512 Text 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,MEOW_PPOW,0,512;area "area" true true false 8 Double 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,area,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,ORIG_FID,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,MPA_dis_Eco_int_singl_Replic2_1,Shape_Area,-1,-1',
        sort_field=None
    )
    
    # Export table to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = out_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
select_big_excel()

In [20]:
# Convert the output features to a Pandas DataFrame
df = pd.DataFrame.spatial.from_featureclass("MPA_dis_Eco_int_singl_big_Replic2_1")


In [21]:
# Display the DataFrame
df.head()

,OBJECTID,FID_WDPA_MPA_dis,FID_MEco_mol_Carib,ECOREGION,REALM,PROVINC,TYPE,BIOME,MEOW_PPOW,area,ORIG_FID,SHAPE
0,1,3489,15,Guianan,Tropical Atlantic,North Brazil Shelf,MEOW,,Guianan,3.529774,6,"{""hasZ"": true, ""rings"": [[[-6047228.252800001,..."
1,2,3508,15,Guianan,Tropical Atlantic,North Brazil Shelf,MEOW,,Guianan,7.753426,25,"{""hasZ"": true, ""rings"": [[[-6032748.834799999,..."
2,3,3514,15,Guianan,Tropical Atlantic,North Brazil Shelf,MEOW,,Guianan,1.874949,31,"{""hasZ"": true, ""rings"": [[[-6040808.767100001,..."
3,4,3515,15,Guianan,Tropical Atlantic,North Brazil Shelf,MEOW,,Guianan,2.801874,32,"{""hasZ"": true, ""rings"": [[[-6041959.022399999,..."
4,5,3517,15,Guianan,Tropical Atlantic,North Brazil Shelf,MEOW,,Guianan,0.462409,34,"{""hasZ"": true, ""rings"": [[[-6042892.1697, 1117..."


In [22]:
# Make pivot tables
pivot_MPA_ecoregion = df.pivot_table(
     index = "MEOW_PPOW",
     values = "OBJECTID",
     aggfunc = "count"
).reset_index()
pivot_MPA_ecoregion.columns = ["MEOW_PPOW", "count"]
print(pivot_MPA_ecoregion)



                      MEOW_PPOW  count
0                      Amazonia      2
1                      Bahamian     74
2                    Carolinian      7
3             Eastern Caribbean     57
4           Equatorial Atlantic      2
5                     Floridian     45
6              Greater Antilles    145
7                       Guianan     70
8                   Gulf Stream      1
9           Inter American Seas     54
10  North Central Atlantic Gyre     10
11      Northern Gulf of Mexico    114
12           Southern Caribbean     48
13      Southern Gulf of Mexico     24
14       Southwestern Caribbean     83
15            Western Caribbean     40


In [23]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Replication"
table = pivot_MPA_ecoregion
output_name = "MPA_dis_Eco_int_singl_big_Replic2_1_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

Reminder to add in the marine ecoregions/pelagic provinces that fall within the study area but don't have any intersections with MPAs so they dont show up in the results. I think there are 3 or so of them.